<a href="https://colab.research.google.com/github/cherylberger/Big-Data/blob/main/Tokenize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tokens").getOrCreate()

In [4]:
from pyspark.ml.feature import Tokenizer

In [6]:
# Create sample DataFrame
dataFrame = spark.createDataFrame([
                                   (0, "Spark is Great"),
                                   (1, " We are learning Spark"),
                                   (2, "Spark is better than Hadoop, no doubt")
], ["id", "sentence"])
dataFrame.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|      Spark is Great|
|  1| We are learning ...|
|  2|Spark is better t...|
+---+--------------------+



In [7]:
# Tokenize sentences
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenizer

Tokenizer_26ddd4ca1f47

In [9]:
tokenized_df = tokenizer.transform(dataFrame)
tokenized_df.show(truncate=False)

+---+-------------------------------------+---------------------------------------------+
|id |sentence                             |words                                        |
+---+-------------------------------------+---------------------------------------------+
|0  |Spark is Great                       |[spark, is, great]                           |
|1  | We are learning Spark               |[, we, are, learning, spark]                 |
|2  |Spark is better than Hadoop, no doubt|[spark, is, better, than, hadoop,, no, doubt]|
+---+-------------------------------------+---------------------------------------------+



In [10]:
# Create a function to return the length of a list
def word_list_length(word_list):
    return len(word_list)

In [11]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [13]:
# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

In [17]:
# Create our tokenizer
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

#Transform DataFrame
tokenized_df = tokenizer.transform(dataFrame)

#Select the needed columns and dont truncate the results
tokenized_df.withColumn("tokens", count_tokens(col("words"))).show(truncate=False)

+---+-------------------------------------+---------------------------------------------+------+
|id |sentence                             |words                                        |tokens|
+---+-------------------------------------+---------------------------------------------+------+
|0  |Spark is Great                       |[spark, is, great]                           |3     |
|1  | We are learning Spark               |[, we, are, learning, spark]                 |5     |
|2  |Spark is better than Hadoop, no doubt|[spark, is, better, than, hadoop,, no, doubt]|7     |
+---+-------------------------------------+---------------------------------------------+------+

